In [1]:
from args import SNNArgs
from model.textcnn import TextCNN
from main import *
from snntorch import surrogate

arg = SNNArgs()
build_environment(arg)
model = TextCNN(arg, spike_grad=surrogate.fast_sigmoid).to(arg.device)

print(list(model._modules.values()))

/home/xujh/miniconda3/envs/snn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[ModuleList(
  (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
  (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
), ModuleList(
  (0): MaxPool2d(kernel_size=(23, 1), stride=(23, 1), padding=0, dilation=1, ceil_mode=False)
  (1): MaxPool2d(kernel_size=(22, 1), stride=(22, 1), padding=0, dilation=1, ceil_mode=False)
  (2): MaxPool2d(kernel_size=(21, 1), stride=(21, 1), padding=0, dilation=1, ceil_mode=False)
), Leaky(), Linear(in_features=300, out_features=2, bias=True), Leaky()]


In [8]:
# for c in model.convs_1:
#     print(c)
spikes = torch.tensor([0,1,1,1,1,1,0])
zeros = torch.eq(spikes, 0)
print(zeros)
print(torch.sum(zeros))
# print(zeros.shape[0].item())
print(7/2)
l = [0,2,3,4,5]
print(", ".join(["spike no.{}: dead ratio:{}".format(index, value) 
            for index, value in enumerate(l)]))

tensor([ True, False, False, False, False, False,  True])
tensor(2)
3.5
spike no.0: dead ratio0, spike no.1: dead ratio2, spike no.2: dead ratio3, spike no.3: dead ratio4, spike no.4: dead ratio5


In [1]:
t = torch.tensor([1,2,3]).to("cuda")
print(t)
a = t.clone().detach()
t.add_(1)
print(a.shape == t.shape)
print(t)

NameError: name 'torch' is not defined